In [1]:
#export
from pytex.core.Precedence import PRECEDENCE
from pytex.LatexInterface.LatexFormatters import LatexBasicFormatter
from pytex.LatexInterface import Operator
from functools import partial
from pytex.LatexInterface.LatexFormatters import LatexSeriesFormatter, LatexDerivativesFormatter

In [2]:
#export
class BasicOperator:
    def __init__(self, precedence, combine):
         self._precedence, self._combine = precedence, combine
    def __call__(self, *args):
        return self._combine(*args)
    @property
    def precedence(self):
        return self._precedence

In [3]:
#export
class InvertibleOperator(BasicOperator):
    def __init__(self, precedence, combine, invert_combine=None):
        super().__init__(precedence, combine)
        self._invert_combine = invert_combine
        
    def __call__(self, *args):
        return super().__call__(*args)
    
    def __invert__(self):
        if self._invert_combine is not None:
            return InvertibleOperator(self.precedence, self._invert_combine, self._combine)
        else:
            return self

def _infix_combine(op):
    return partial(LatexBasicFormatter.binary_operation_output, op)

class InvertibleInfixOperator(InvertibleOperator):
    def __init__(self, precedence, op, invert_op=None):
        c, invc = _infix_combine(op), _infix_combine(invert_op)
        if invert_op is None: 
            super().__init__(precedence, c)
        else: 
            super().__init__(precedence, c, invc)

In [4]:
#export
Plus = InvertibleInfixOperator(PRECEDENCE.ARITHMATIC_ADD, Operator.plus)
Minus = InvertibleInfixOperator(PRECEDENCE.ARITHMATIC_ADD, Operator.minus)
Equal = InvertibleInfixOperator(PRECEDENCE.EQUALITY, Operator.eq, Operator.neq)
LessThan = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.lt, Operator.nless)
LessThanEqual = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.leq, Operator.nleq)
GreaterThan = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.gt, Operator.ngtr)
GreaterThanEqual = InvertibleInfixOperator(PRECEDENCE.COMPARE, Operator.geq, Operator.ngeq)
Space = InvertibleInfixOperator(PRECEDENCE.EQUALITY, Operator.space)

In [5]:
#export
Negate = InvertibleOperator(PRECEDENCE.POWER, 
                            LatexBasicFormatter.negate, 
                            LatexBasicFormatter.unnegate)

In [6]:
(~Negate)('-1')

'1'

In [36]:
#export
mult_combine = _infix_combine(Operator.multiply)
frac_combine = LatexBasicFormatter.fraction

Multiply = InvertibleOperator(PRECEDENCE.ARITHMATIC_MULT, mult_combine, frac_combine)
Fraction = ~Multiply

In [37]:
#export
power_combine = _infix_combine(Operator.power)

Power = InvertibleOperator(PRECEDENCE.POWER, power_combine, LatexBasicFormatter.root)
Root = ~Power

In [38]:
print(Root('f', 'a'), Power('f','a'))

\sqrt[\leftroot{-1}\uproot{1}a]{f} f ^ a


In [39]:
#export
f_combine = LatexBasicFormatter.function_definition
inv_f_combine = LatexBasicFormatter.inverse_function_definition

FunctionCall = InvertibleOperator(PRECEDENCE.POWER, f_combine, inv_f_combine)

In [10]:
#export
Summation = InvertibleOperator(PRECEDENCE.POWER, 
                               LatexSeriesFormatter.summation)

Product = InvertibleOperator(PRECEDENCE.POWER,
                             LatexSeriesFormatter.product)

In [40]:
#export
def inv_integral(function, wrt, lower='', upper=''):
    return LatexDerivativesFormatter.derivative_of(function, wrt)

Integral =  InvertibleOperator(PRECEDENCE.POWER, 
                               LatexSeriesFormatter.integral, 
                               inv_integral)
Derivative = ~Integral

def inv_partial(function, wrt, lower='', upper=''):
    return LatexDerivativesFormatter.partial_of(function, wrt)

PartialIntegral = InvertibleOperator(PRECEDENCE.POWER,
                                     LatexSeriesFormatter.partial_integral, 
                                     inv_partial)
PartialDerivative = ~PartialIntegral

In [41]:
print((Derivative)('f', 'x', '1', '2'))

\frac{df}{dx}


$$
1 \neq 2 \\
\sqrt[\leftroot{-1}\uproot{1}b]{a} \\
a ^ b
$$

In [42]:
! python3 ../notebook2script.py Operator.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/core/Operator.ipynb to /home/default/Documents/py-latex/pytex/pytex/core/Operator.py
